In [ ]:
# import sys
# import os

# # Specify the path to the directory containing residPCA.py
# script_path = "/Users/shayecarver/residPCA/src/residPCA"

# # Add the path to the system path
# sys.path.append(os.path.abspath(script_path))

from residPCA import *

In [ ]:
scExp = residPCA(count_matrix_path="/Users/shayecarver/residPCA/examples/example_data.h5ad", 
               vars_to_regress=['Batch','celltype','total_counts', 'pct_counts_mt','Age','Sex'],
               object_columns=['celltype','Sex','Batch'], 
               save_image_outputs=True, 
               BIC=True, 
               vargenes_Stand_resid=3000,
               n_PCs = 150,
               random_seed=7, 
               vargenes_IterPCA=3000, 
               #path_to_directory = "./", 
               #basename=f'Morab_RNA_3kvargenes_seurat',
               variable_genes_flavor="seurat",  
               global_ct_cutoff=0.2)  
scExp.Normalize() 
scExp.Standardize()
scExp.StandardPCA_fit()
scExp.residPCA_fit()
scExp.Iter_PCA_fit()
# identify global versus cell type specific states
scExp.ID_Global_CellType_States()
#write BED files for heritability analysis
scExp.heritability_bed_output("~/residPCA/gencode.v39.basic.annotation.names.bed", 200, "Resid")
scExp.heritability_bed_output("~/residPCA/gencode.v39.basic.annotation.names.bed", 200, "Resid")
scExp.heritability_bed_output("~/residPCA/gencode.v39.basic.annotation.names.bed", 200, "Resid")


Directory './ResidPCA_run_2024-12-15_17_20_36' created successfully.
Finding most variable genes using 'seurat' flavor.
Using "seurat" flavor to compute variable genes in Iterative PCA on MG.
Using "seurat" flavor to compute variable genes in Iterative PCA on OPC.
